In [ ]:
''' In the context of this project:
Contains some of the original code for analysis, however it is outdated.
- M'''

In [ ]:
# Post-processing with conformal prediction, using the logits tracked from training
import os
import numpy as np
import pandas as pd
import glob
import re
from scipy.special import softmax

from scipy.io import loadmat
from skimage.transform import resize
from sklearn.metrics import f1_score, confusion_matrix
import matplotlib.pyplot as plt

In [ ]:
# analyze the dataframes for training and validation from each epoch
# we start by grabbing the dataframes

#csv_folder = "../logs/Baseline_r2plus1d_all/csvs/"
csv_folder = "../logs/R18_cifar/round1/csvs"
train_csvs = glob.glob(f"{csv_folder}/train_*.csv")
val_csvs = glob.glob(f"{csv_folder}/val_*.csv")

N_CLASSES=10
ALPHA=0.10

def extract_file_index(filename):
    # Use regular expression to find numerical part in the filename
    match = re.findall(r'\d+', filename)[-3]
    if match:
        numerical_part = int(match)
        return numerical_part
    else:
        # Return a default value or handle the case where no numerical part is found
        return None

# sort ascending by the numerical suffix for each file with regex search
def sort_by_epoch(files_list):
    indices = []
    for i in range(len(files_list)):
        index = extract_file_index(files_list[i])
        indices.append(index)
    new_array = [None] * len(indices)
    for j in range(len(indices)):
        new_array[indices[j]] = files_list[j]
    return new_array

print(sort_by_epoch(val_csvs))
#print(train_csvs)

In [ ]:
train_df = [pd.read_csv(x) for x in sort_by_epoch(train_csvs)]
val_df = [pd.read_csv(x) for x in sort_by_epoch(val_csvs)]

# since train_df is ordered randomly, sort each dataframe by the filename
train_df = [df.sort_values(by=['filename']).reset_index() for df in train_df]
val_df = [df.sort_values(by=['filename']).reset_index() for df in val_df]

In [ ]:
train_df[0]

In [ ]:
def df_to_logits(df, sm=False):
    # take the outputs_x rows and conver them to array of (N, C)
    cols = [x for x in df.columns if 'outputs' in x]
    logits = df[cols].to_numpy()
    if sm:
        logits = softmax(logits, axis=1)
    return logits

train_probs = [df_to_logits(x, False) for x in train_df]
val_probs = [df_to_logits(x, False) for x in val_df]

In [ ]:
# show the trend-line of prediction F1 score for training and test set
def plot_f1_over_time(epoch_0_dataframe, all_probs):
    # epoch_0_dataframe is the dataframe results at epoch 0
    # probs_progression is the list of probabilities for each epoch
    gt = np.array(epoch_0_dataframe['y'])
    preds = [np.argmax(p, axis=1) for p in all_probs]
    f1s = [f1_score(gt, p, average='macro') for p in preds]
    fig, ax = plt.subplots(figsize=(12,4))
    t = np.arange(len(preds))
    ax.plot(t, f1s)
    plt.show()

def cf_over_time(epoch_0_dataframe, all_probs):
    gt = np.array(epoch_0_dataframe['y'])
    preds = [np.argmax(p, axis=1) for p in all_probs]
    [print(confusion_matrix(gt, p)) for p in preds]

plot_f1_over_time(train_df[0], train_probs)
plot_f1_over_time(val_df[0], val_probs)
cf_over_time(train_df[0], train_probs)
cf_over_time(val_df[0], val_probs)

In [ ]:
# we are setting up the validation set as the calibration set, to be more kosher, you should use an unseen subset of training
# check the certainty distribution of validation set
def certainty_hist(y, preds, epoch=-1):
    # y is (N,) integer array of C classes
    # preds is (N, C) array
    pred_classes = np.argmax(preds,axis=1)
    pred_confidence = np.max(preds,axis=1)
    gt_confidence = preds[range(len(y)),y]
    print(f"Epoch {epoch}, F1 score = {f1_score(y, pred_classes, average='macro')}")
    fig, ax = plt.subplots(figsize=(8,3))
    ax.hist([gt_confidence, pred_confidence], bins=49)
    plt.legend(['GT', 'argmax'])
    plt.title(f"Epoch {i} label certainty distribution")
    plt.show()

# first check the validation set for naive softmaxing
for i in range(len(val_df)):
    y = val_df[i]['y']
    preds = softmax(val_probs[i], axis=1)
    certainty_hist(y, preds, i)

In [ ]:
# run conformal prediction on a select epoch to return the prediction sets
def LABEL(y, preds, alpha=0.1, verbose=False):
    # y is (N,) integer array of C classes
    # preds is (N, C) array of confidences
    
    desired_accuracy = 1 - alpha
    N = len(y)
    pred_classes = np.argmax(preds,axis=1)
    pred_confidence = np.max(preds,axis=1)
    gt_confidence = preds[range(N),y]
    # calculate the conformal score s as a general imprecise uncertainty measure (can be many ways, this is a simple way)
    s = 1 - gt_confidence
    # find the q level
    adj_q_level = np.ceil((N+1) * desired_accuracy)/N
    #adj_q_level = desired_accuracy
    qhat = np.quantile(s, adj_q_level)
    # create sets based on the q-level
    cutoff = 1 - qhat
    conformal_set = []
    for i in range(N):
        conformal_set.append(preds[i] >= cutoff)
        
    if verbose:
        print(f"adjusted q-level {adj_q_level}, q-hat {qhat}")
    return np.array(conformal_set)

In [ ]:
def APS(cal_labels, cal_smx, alpha=0.1, verbose=False):
    # Get scores. calib_X.shape[0] == calib_Y.shape[0] == n
    N = cal_smx.shape[0]
    
    # argsort returns the indices of the sort
    cal_pi = cal_smx.argsort(1)[:, ::-1]
    
    # take_along_axis returns results of the sort, then cumulative sum
    cal_srt = np.take_along_axis(cal_smx, cal_pi, axis=1).cumsum(axis=1)
    
    # this is some 5head stuff I dont get
    # but it returns "the cumulative sum by the time I get to class X"
    cal_scores = np.take_along_axis(cal_srt, cal_pi.argsort(axis=1), axis=1)[range(N), cal_labels]
    
    # Get the score quantile
    adj_q_level = np.ceil((N + 1) * (1 - alpha)) / N
    qhat = np.quantile(cal_scores, adj_q_level, interpolation="higher")
    if verbose:
        print(f"adjusted q-level {adj_q_level}, q-hat {qhat}")
    
    # Deploy (output=list of length n, each element is tensor of classes)
    cal_pi = cal_smx.argsort(1)[:, ::-1]
    
    cal_srt = np.take_along_axis(cal_smx, cal_pi, axis=1).cumsum(axis=1)
    
    prediction_sets = np.take_along_axis(cal_srt <= qhat, cal_pi.argsort(axis=1), axis=1)
    return prediction_sets
    

In [ ]:
y = val_df[2]['y']
preds = softmax(val_probs[2], axis=1)
cs = APS(y, preds, ALPHA, verbose=True)
print(cs[:15])

In [ ]:
# test the coverage of conformal prediction
def coverage_test(y, preds_cover):
    # y is (N,) integer array of C classes
    # preds_cover is (N, C) boolean array indicating the cover
    correct = []
    for i in range(len(y)):
        if preds_cover[i, y[i]]:
            correct.append(True)
        else:
            correct.append(False)
    return np.array(correct)

correctness = coverage_test(y, cs) 
print(np.sum(correctness)/len(y)) # should be around 0.9

In [ ]:
def class_conditional_coverage_test(y, preds_cover, num_classes=4):
    # returns an accuracy for each class
    class_cond_acc = np.zeros(num_classes)
    for c in range(num_classes):
        mask = y == c
        preds_c = preds_cover[mask]
        N_c = len(preds_c)
        correct = 0
        for i in range(N_c):
            if preds_c[i,c]:
                correct += 1
        class_cond_acc[c] = correct/N_c
    return class_cond_acc
correctness = class_conditional_coverage_test(y, cs, N_CLASSES) 
correctness

In [ ]:
# check the cardinality of the conformal prediction
def cp_cardinality(y, preds_cover, preds, n_classes=4):
    cardinality = np.sum(preds_cover, axis=1)
    correct = coverage_test(y, preds_cover)
    card_correct = cardinality[correct]
    card_incorrect = cardinality[~correct]
    
    n_bins = n_classes + 1
    c_amount, c_bins = np.histogram(card_correct, bins=np.arange(n_bins)+0.5)
    n_amount, _ = np.histogram(card_incorrect, bins=np.arange(n_bins)+0.5)
    card_acc = c_amount / (c_amount + n_amount + 1e-9)
    print(f"Number of correct preds of size [1..N_classes]: {c_amount}")
    print(f"Number of incorrect preds of size [1..N_classes]: {n_amount}")
    print(f"Accuracy of preds of size [1..N_classes]: {card_acc}")

    top1_accs = []
    pred_classes = np.argmax(preds, axis=1)
    for i in np.arange(n_classes)+1:
        gt_w_card_i = y[cardinality==i]
        pred_w_card_i = pred_classes[cardinality==i]
        acc_w_card_i = np.sum(gt_w_card_i == pred_w_card_i) / (len(gt_w_card_i) + 1e-9)
        top1_accs.append(acc_w_card_i)
    print(f"Top-1 acc of preds of size [1..N_classes]: {top1_accs}")
    return np.mean(cardinality)

cp_cardinality(y, cs, preds, n_classes=N_CLASSES)

In [ ]:
# run conformal prediction on each epoch
f1s = []
coverages = []
cardinalities = []
for i in range(len(val_df)):
    y = np.array(val_df[i]['y'])
    preds = softmax(val_probs[i], axis=1)
    pred_classes = np.argmax(preds,axis=1)
    print(f"Epoch {i}")
    print(f"Original classifier F1 = {f1_score(y, pred_classes, average='macro')}")
    f1s.append(f1_score(y, pred_classes, average='macro'))
    
    #cs = LABEL(y, preds, ALPHA)
    cs = APS(y, preds, ALPHA)
    
    correctness = coverage_test(y, cs)
    print(f"Conformal coverage accuracy = {np.sum(correctness)/len(y)}")
    coverages.append(np.sum(correctness)/len(y))
    correctness = class_conditional_coverage_test(y, cs, num_classes=N_CLASSES)
    print(f"Class-conditional coverage accuracy = {correctness}")
    avg_card = cp_cardinality(y, cs, preds, n_classes=N_CLASSES)
    cardinalities.append(avg_card)

In [ ]:
plt.plot(range(len(val_df)), f1s)
plt.plot(range(len(val_df)), coverages)
plt.plot(range(len(val_df)), np.full(shape=len(val_df), fill_value=0.90))
plt.title('Label evolution: validation set top-1 F1 vs coverage over training')
plt.legend(['top-1 F1 score', 'coverage acc.', 'theoretical coverage acc.'])
plt.show()

plt.figure()
plt.plot(range(len(val_df)), cardinalities)
plt.title('Coverage cardinality over time')
plt.legend(['Cardinality'])
plt.show()